[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_dnam.ipynb) [![Open In nbviewer](https://img.shields.io/badge/View%20in-nbviewer-orange)](https://nbviewer.jupyter.org/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_dnam.ipynb)

# Bulk DNA methylation

This tutorial is a brief guide for the implementation of an array of bulk DNA-methylation epigenetic clocks. In this notebook, we will demonstrate the breadth of epigenetic clock models available in *pyaging* by showing:

- Horvath's 2013 ElasticNet-based clock ([url](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2013-14-10-r115));
  
- AltumAge, a highly accurate deep-learning based clock ([url](https://www.nature.com/articles/s41514-022-00085-y));
  
- PCGrimAge, a principal-component based version of the GrimAge clock ([url](https://www.nature.com/articles/s43587-022-00248-2));

## Import packages

We just need two packages for this tutorial.

In [1]:
import pandas as pd
import pyaging as pya

## Download and load example data

Let's download the publicly avaiable dataset GSE139307 with Illumina's 450k array. The CpG coverage of the 450k array should be good enough for most clocks.

In [2]:
pya.data.download_example_data('methylation')

|-----> 🏗️ Starting download_example_data function
|-----> ⚙️ Download data started
|-----------> Data found in ./pyaging_data/GSE139307.pkl
|-----> ✅ Download data finished [0.0009s]
|-----> 🎉 Done! [0.0019s]


In [3]:
df = pd.read_pickle('pyaging_data/GSE139307.pkl')

In [4]:
df.head()

,dataset,tissue_type,age,gender,cg00000029,cg00000108,cg00000109,cg00000165,cg00000236,cg00000289,...,ch.X.93511680F,ch.X.938089F,ch.X.94051109R,ch.X.94260649R,ch.X.967194F,ch.X.97129969R,ch.X.97133160R,ch.X.97651759F,ch.X.97737721F,ch.X.98007042R
GSM4137709,GSE139307,sperm,84.0,M,0.084811,0.920696,0.856851,0.084567,0.838699,0.247273,...,0.061751,0.045942,0.037631,0.056455,0.249872,0.049022,0.085691,0.037435,0.077820,0.106234
GSM4137710,GSE139307,sperm,69.0,M,0.099626,0.919073,0.890024,0.115541,0.852584,0.198103,...,0.075077,0.041849,0.032573,0.089790,0.250245,0.079095,0.079756,0.046229,0.091256,0.120241
GSM4137711,GSE139307,sperm,69.0,M,0.117228,0.920276,0.894317,0.117127,0.839258,0.213410,...,0.068679,0.049515,0.058097,0.079919,0.299758,0.079305,0.089815,0.065364,0.086864,0.156005
GSM4137712,GSE139307,sperm,69.0,M,0.077096,0.910204,0.908400,0.073885,0.861615,0.163276,...,0.070091,0.033289,0.038836,0.108213,0.295428,0.050731,0.099943,0.047597,0.078480,0.107480
GSM4137713,GSE139307,sperm,67.0,M,0.063524,0.911608,0.884643,0.079877,0.864654,0.176169,...,0.082368,0.038411,0.048787,0.088631,0.316694,0.041873,0.079303,0.048823,0.089010,0.117903


For PCGrimAge, both age and sex are features. Therefore, to get the full prediction, let's convert the column `gender` into a column called `female`, with 1 being female and 0 being male. Let's just drop the other metadata for simplicity.

In [5]:
# needs only numerical data (doesn't work with strings)
df['female'] = (df['gender'] == 'F').astype(int)
df = df.drop(['gender', 'tissue_type', 'dataset'], axis=1)

## Convert data to AnnData object

AnnData objects are highly flexible and are thus our preferred method of organizing data for age prediction.

In [6]:
adata = pya.pp.df_to_adata(df, imputer_strategy='mean')

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Impute missing values started
|-----------> Imputing missing values using mean strategy
|-----> ✅ Impute missing values finished [0.1123s]
|-----> ⚙️ Log data statistics started
|-----------> There are 37 observations
|-----------> There are 485514 features
|-----------> Total missing values: 0
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.2125s]
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0150s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0320s]
|-----> ⚙️ Add unstructured data to anndata started
|-----> ✅ Add unstructured data to anndata finished [0.0012s]
|-----> 🎉 Done! [0.3736s]


## Predict age

We can either predict one clock at once or all at the same time. For convenience, let's simply input all three clocks of interest at once. The function is invariant to the capitalization of the clock name. 

In [7]:
adata = pya.pred.predict_age(adata, ['Horvath2013', 'AltumAge', 'PCGrimAge',])

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0005s]
|-----> Processing clock: Horvath2013
|-----------> ⚙️ Load clock started
|-----------> ⚙️ Download data started
|-----------> Data found in ./pyaging_data/horvath2013.pt
|-----------> ✅ Download data finished [0.0004s]
|-----------> ✅ Load clock finished [0.0004s]
|-----------> ⚙️ Check features in adata started
|-----------> All features are present in adata.var_names.
|-----------> ✅ Check features in adata finished [0.0036s]
|-----------> ⚙️ Convert adata.X to torch.tensor and filter features started
|-----------> ✅ Convert adata.X to torch.tensor and filter features finished [0.0016s]
|-----------> ⚙️ Initialize model started
|-----------> ✅ Initialize model finished [0.0024s]
|-----------> ⚙️ Predict ages with model started
|-----------> ✅ Predict ages with model finished [0.0015s]
|-----------> ⚙️ Convert tensor to numpy a

In [8]:
adata.obs.head()

,horvath2013,altumage,pcgrimage
GSM4137709,33.624774,37.007023,68.520729
GSM4137710,28.829343,29.426702,58.770798
GSM4137711,28.316548,22.805416,57.613228
GSM4137712,24.850635,18.060070,59.368393
GSM4137713,25.942114,20.071888,59.199066


## Get citation

The doi, citation, and some metadata are automatically added to the AnnData object under `adata.uns[CLOCKNAME_metadata]`.

In [9]:
adata.uns['horvath2013_metadata']

{'species': 'Homo sapiens',
 'data_type': 'methylation',
 'year': 2013,
 'citation': 'Horvath, Steve. "DNA methylation age of human tissues and cell types." Genome biology 14.10 (2013): 1-20.',
 'doi': 'https://doi.org/10.1186/gb-2013-14-10-r115'}

In [10]:
adata.uns['altumage_metadata']

{'species': 'Homo sapiens',
 'data_type': 'methylation',
 'year': 2022,
 'citation': 'de Lima Camillo, Lucas Paulo, Louis R. Lapierre, and Ritambhara Singh. "A pan-tissue DNA-methylation epigenetic clock based on deep learning." npj Aging 8.1 (2022): 4.',
 'doi': 'https://doi.org/10.1038/s41514-022-00085-y'}

In [11]:
adata.uns['pcgrimage_metadata']

{'species': 'Homo sapiens',
 'data_type': 'methylation',
 'year': 2022,
 'citation': 'Higgins-Chen, Albert T., et al. "A computational solution for bolstering reliability of epigenetic clocks: Implications for clinical trials and longitudinal tracking." Nature aging 2.7 (2022): 644-661.',
 'doi': 'https://doi.org/10.1038/s43587-022-00248-2'}